In [1]:
!pip install -U numpy hazm accelerate pypdf2 llama-index llama-index-llms-huggingface llama-index-embeddings-huggingface -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 60.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 31.8 MB

In [2]:
!pip install -i https://pypi.org/simple/ bitsandbytes -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.1 MB/s eta 0:00:00


In [3]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

from transformers import AutoTokenizer, AutoModelForCausalLM
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
import torch

import os
from hazm import Normalizer
from PyPDF2 import PdfReader

## Prepare Documents

In [4]:
!unzip -q pdf_documents.zip -d /content/pdf_data

In [5]:
# Directory where PDF files are stored
pdf_dir = 'pdf_data'

# Directory where text files will be stored
text_dir = 'data'

# Ensure the text directory exists
os.makedirs(text_dir, exist_ok=True)

# Initialize hazm Normalizer
normalizer = Normalizer()

In [6]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as f:
        reader = PdfReader(f)
        texts = []
        for i in range(0, len(reader.pages), 5):
            text = ""
            for j in range(i, min(i+5, len(reader.pages))):
                page = reader.pages[j]
                text += page.extract_text()
            texts.append(text)
        return texts


# Convert all PDFs in the pdf_dir to text files in the text_dir
for pdf_file in os.listdir(pdf_dir):
    if pdf_file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_dir, pdf_file)
        texts = extract_text_from_pdf(pdf_path)
        for i, text in enumerate(texts):
            normalized_text = normalizer.normalize(text)
            text_file_path = os.path.join(text_dir, f"{pdf_file.replace('.pdf', '')}_{i+1}.txt")
            with open(text_file_path, 'w', encoding='utf-8') as text_file:
                text_file.write(normalized_text)

print("PDF to text conversion completed.")

PDF to text conversion completed.


## Initiate LLM

In [7]:
MODEL_NAME = "NousResearch/Meta-Llama-3-8B-Instruct"
TOKENIZER_NAME = "NousResearch/Meta-Llama-3-8B-Instruct"

In [8]:
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)

# Define stopping IDs (EOS token)
stopping_ids = [tokenizer.eos_token_id]

generate_kwargs = {
    "do_sample": False,
    "temperature": 0.6,
    "top_p": 0.9,
}

# Load the model with specific parameters
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="cuda",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    load_in_8bit=True
)

# Initialize HuggingFace LLM with the preloaded model and tokenizer
llm = HuggingFaceLLM(
    model=model,
    tokenizer=tokenizer,
    generate_kwargs=generate_kwargs,
    stopping_ids=stopping_ids,
    system_prompt="You are a Persian chatbot capable of answering municipality questions. Answer all questions just in Persian."
)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [9]:
# Initialize embedding model
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/LaBSE")

# Assign to Settings (if necessary)
Settings.embed_model = embed_model
Settings.llm = llm

# Load documents
documents = SimpleDirectoryReader("data").load_data()

# Create VectorStoreIndex from documents
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model, llm=llm)

# Initialize query engine
query_engine = index.as_query_engine(similarity_top_k=3)

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [18]:
llm.system_prompt = "Write answer of questions just in persian!"

In [10]:
# Perform a query
response = query_engine.query("مدارک لازم جهت تشکیل پرونده ساختمان و تخریب نوسازی چیست؟")

print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 برای تشکیل پرونده ساختمان و تخریب نوسازی، مدارک زیر لازم است:

* طرح تفصیلی
* مجوز ساخت و ساز از شهرداری
* مجوز از سازمان میراث فرهنگی، گردشگری و صنایع‌دستی در صورت ساخت و ساز در مجاورت و یا حریم آثار ثبت‌شده و یا واجد ارزش اعالم شده
* مجوز از شهرداری و تأیید سازمان میراث فرهنگی، گردشگری و صنایع‌دستی در صورت تغییر در نمای ابنیه ثبت شده و واجد ارزش اعالم شده تاریخی و فرهنگی و ارزشمند معاصر
* گزارش کمیته اجرایی موضوع مصوبه مندرج در بند ۲ - ۱۳ در صورت ساخت و ساز در مجاورت و یا حریم آثار ثبت‌شده و یا واجد ارزش اعالم شده
* گزارش سازمان میراث فرهنگی، صنایع‌دستی و گردشگری استان تهران در صورت ساخت و ساز در مجاورت و یا حریم آثار ثبت‌شده و یا واجد ارزش اعالم شده

Note: The answer is based on the provided context information and may not be a comprehensive list of required documents for building construction and renovation in Tehran.


## Test with Question1 and Question2 files

In [11]:
!pip install pandas -q

In [12]:
!unzip -q questions.zip -d /content/questions

In [13]:
import pandas as pd

# Path to the uploaded Excel file
excel1_file_path = '/content/questions/question1.xlsx'
excel2_file_path = '/content/questions/question2.xlsx'

excel1 = pd.read_excel(excel1_file_path, sheet_name=0)
excel2 = pd.read_excel(excel2_file_path, sheet_name=0)

df1 = excel1[['پاسخ','سوال']]
df2 = excel2[['پاسخ تفضیلی و تحلیلی','عنوان پرسش']]

# Rename the columns to "question" and "answer"
df1.rename(columns= { 'سوال': 'Question', 'پاسخ': 'Answer' }, inplace=True)
df2.rename(columns= { 'عنوان پرسش': 'Question', 'پاسخ تفضیلی و تحلیلی': 'Answer' }, inplace=True)

df = pd.concat([df1, df2], ignore_index=True)
df = df.dropna()
df.head()

,Answer,Question
0,با سلام و احترام؛ لطفا به شهرداری منطقه مراجعه...,باسلام اینجانب دارای 300 زمین مشاع در پلاک ثبت...
2,با سلام و احترام؛ صدور پروانه صرفا درچهارچوب ض...,با سلام احترام خدمت شما من سال پیش در منطقه ۱۴...
4,با سلام و احترام؛ در نسخه دیجیتالی اخذ پرینت ...,چگونه می توانم از صورتجلسات کمیسیون ماه 5 پرین...
6,با سلام و احترام تا این تاریخ شیوه نامه مذکور ...,باسلام اینجانب زمین فریز شده در منطقه ۱۴ فریز ...
8,با سلام و احترام؛ هرساختمانی که احداث می شود...,با سلام مجدد بنده از همکاران شهرداری میباشم قب...


In [19]:
# Function to get LLM response for each question
def get_llm_response(question):
    question = normalizer.normalize(question)
    response = query_engine.query(question)
    return response

In [22]:
chosen_df = df.iloc[40:75]

In [23]:
# Apply the function to each question and store the result in a new column
from tqdm import tqdm

tqdm.pandas()

tqdm.pandas(desc="Processing rows")
chosen_df['Model Answer'] = chosen_df['Question'].progress_apply(get_llm_response)

Processing rows:  46%|████▌     | 16/35 [13:14<16:08, 50.99s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Processing rows: 100%|██████████| 35/35 [28:19<00:00, 48.56s/it]


In [24]:
chosen_df.to_csv('questions_40_75.csv', index=False, encoding='utf-8')

# Evaluation

In [25]:
from sentence_transformers import SentenceTransformer

evaluator_model = SentenceTransformer('Alibaba-NLP/gte-large-en-v1.5', trust_remote_code=True)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/71.2k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

configuration.py:   0%|          | 0.00/7.13k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [26]:
!unzip -q outputs.zip

In [30]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Function to get list of all CSV files in a directory
def get_csv_files(directory):
    return [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.csv')]

# Function to merge all CSV files into a single DataFrame
def merge_csv_files(files):
    dataframes = []
    for file in files:
        df = pd.read_csv(file)
        dataframes.append(df)
    merged_df = pd.concat(dataframes, ignore_index=True)
    return merged_df

# Directory containing the CSV files
directory = 'outputs'

# Get list of CSV files
csv_files = get_csv_files(directory)

# Merge all CSV files
merged_df = merge_csv_files(csv_files)

In [31]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Answer        139 non-null    object 
 1   Question      139 non-null    object 
 2   Model Answer  139 non-null    object 
 3   similarity    40 non-null     float64
 4   Similarity    40 non-null     float64
dtypes: float64(2), object(3)
memory usage: 5.6+ KB


In [32]:
# Function to calculate similarity
def calculate_similarity(answer, model_answer):
    # Encode sentences to get their embeddings
    embeddings1 = evaluator_model.encode(answer, convert_to_tensor=True)
    embeddings2 = evaluator_model.encode(model_answer, convert_to_tensor=True)

    # Compute cosine similarity
    similarity_score = util.pytorch_cos_sim(embeddings1, embeddings2)

    return similarity_score.item()

## First Model

In [33]:
# Add a new column for similarity scores
merged_df['Similarity'] = 0.0

# Compute the total score
total_score = 0
for index, row in tqdm(merged_df.iterrows(), total=merged_df.shape[0], desc="Calculating Similarities"):
    answer = row['Answer']
    model_answer = row['Model Answer']
    similarity = calculate_similarity(answer, model_answer)
    merged_df.at[index, 'Similarity'] = similarity
    total_score += similarity

print(f"\nTotal similarity score: {total_score/len(merged_df)}")

# Save the updated DataFrame to a new CSV file
merged_df.to_csv('final_result.csv', index=False)

Calculating Similarities: 100%|██████████| 139/139 [00:32<00:00,  4.30it/s]


Total similarity score: 0.6392572406384585


In [34]:
merged_df

,Answer,Question,Model Answer,similarity,Similarity
0,در خصوص تداخل و تعارض املاک شهرداری، صرفا به ...,شهرداری در خصوص تداخل املاک چگونه اظهار نظر می...,"According to the provided context, it seems t...",NaN,0.484513
1,در صورتی که اطراف ملک در نقشه های پوششی، نهر ...,در چه صورتی از اداره آب استعلام میگردد؟,در این سند، از اداره ثبت اسناد استعلام می‌گرد...,NaN,0.591895
2,در صورتی که اطراف ملک در نقشه های پوششی دکل و...,در چه صورتی از اداره برق استعلام می گردد ؟,۱۳ تعمیرگاه‌های لوازم خانگی، آشپزخانه، برق‌ی و...,NaN,0.670718
3,رای کمیسیون ماده ۱۲ در تعیین نوع اراضی از سوی...,رای کمیسیون ماده ۱۲ توسط چه سازمانی صادر میگردد؟,_____________________________________________...,NaN,0.474264
4,تا ارائه سند شش‌دانگ مالکیت، هیچ گونه اقدامی ...,در خصوص املاکی که سند مشاعی ارائه مینمایند چه ...,در خصوص املاکی که سند مشاعی ارائه می‌نمایند، ...,NaN,0.666510
...,...,...,...,...,...
134,کلیه مسئولیت های مربوط به جابجائی ملک و احداث ...,مسئولیت جابجائی ملک و جابجائی در احداث بنا به ...,According to the provided context information...,NaN,0.506947
135,پس از تشکیل پرونده در شهرداری و بررسی مطابق ض...,تفکیک و افراز املاک چگونه است؟,"According to the context information, تفکیک و...",NaN,0.587974
136,حداقل عرض معابر دسترسي در اجراي طر حهاي جامع و...,حداقل عرض معابر چقدر است؟,۶ متر است. (The minimum width of the sidewalks...,NaN,0.596749
137,در زمان ارائه نقشه های معماری مهندس معمار می ...,کنترل کف چه زمانی باید صورت گیرد؟,در صورتی که سطح اشغال زیرزمین بیشژتر از سژطح ...,NaN,0.557402


## Second Model

In [ ]:
# Add a new column for similarity scores
merged_df['Similarity'] = 0.0

# Compute the total score
total_score = 0
for index, row in tqdm(merged_df.iterrows(), total=merged_df.shape[0], desc="Calculating Similarities"):
    answer = row['Answer']
    model_answer = row['Model Answer']
    similarity = calculate_similarity(answer, model_answer)
    merged_df.at[index, 'Similarity'] = similarity
    total_score += similarity

print(f"\nTotal similarity score: {total_score/len(merged_df)}")

# Save the updated DataFrame to a new CSV file
merged_df.to_csv('final_result.csv', index=False)

Calculating Similarities: 100%|██████████| 139/139 [00:27<00:00,  4.98it/s]


Total similarity score: 0.6586930488082144


In [ ]:
merged_df

,Answer,Question,Model Answer,Similarity
0,هرساختمانی که احداث می شود، می بایست طبق ضواب...,میزان و نحوه تامین پارکینگ بیمارستانهای دولتی،...,"According to the regulations, all hospitals, ...",0.473896
1,صورتجلسات کمیسیون ماده 5 در بخش خدمات و آرش...,چگونه می توان به صورتجلسات کمیسیون ماده 5 دستر...,"According to the provided context, it seems t...",0.616450
2,سوال مطرح شده مرتبط با حوزه شهرسازی و معماری ن...,گردش کار جهت اخذ عوارض نوسازی چیست؟,با توجه به سند، ضوابط و مقررات استفاده از ارا...,0.678449
3,اصل و کپی سند مالکیت با اجاره نامه، اصل و کپی...,مدارک مورد نیاز جهت اخذ عوارض کسب و پیشه واحده...,"According to the provided context, there is n...",0.473872
4,طبق قانون مالکیت در آپارتمان‌ها بر دو بخش تقسی...,منظور از مشاعات ساختمان چیست؟,مشاعات ساختمان، فضاهای غیرمفید و غیرمسکونی سا...,0.695940
...,...,...,...,...
134,بر اساس شاخص های شورای‌عالی شهرسازی و معماری ا...,تعریف بافت فرسوده و ناپایدار چیست ؟,بافت فرسوده و ناپایدار به بافت شهری گفته می‌ش...,0.644207
135,اگر شهرداری‌های مناطق بیست و دوگانه پرونده مب...,آیا کمیسیون ماده 100 میتواند به ساختمان های در...,(کمیسیون ماده ۵ می‌تواند به ساختمان‌های درمان...,0.613213
136,موضوع مورد شکایت می بایست توسط کارشناسان شهرسا...,در صورت عدم رعایت درز انقطاع و سایر تخلفات ساخ...,در صورت عدم رعایت درز انقطاع و سایر تخلفات سا...,0.661369
137,ضوابط پهنه S222 بشرح ذیل میباشد : پهنه مزبور ...,کاربری و تراکم ملک با پهنه s۲۲۲ چگونه است؟,"According to the provided context, پهنه S ۲۲۲...",0.620743
